In [ ]:
import aextl
import numpy
import os
import yaml
import math

import scipy
import scipy.stats

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
%matplotlib inline

In [ ]:
experimentDir = "../experiments/"
exactValueDir = "../experiments/output/brandes/"
#suffix = '-1t'
suffix = ''
selectedVariant = 'ips4375'
selectedInstSet = 'evaluation'

## Read in Experiment Manifest File

In [ ]:
cfg = aextl.config_for_dir(experimentDir)

# Read in Experimental Results

In [ ]:
# Load exact betweenness values for each instance
exactValues = dict()

for instance in cfg.all_instances():
    try:
        with open(os.path.join(exactValueDir, instance.filename + '.out-full'), 'r') as f:
            exactValues[instance.filename] = numpy.loadtxt(f)
        size = len(exactValues[instance.filename])
        normalizingConstant = (size-1)*(size-2)
        for i in range(size):
            exactValues[instance.filename][i] /= normalizingConstant
    except FileNotFoundError as e:
        print(e)

In [ ]:
evaluationInstances = set([instance for instance in cfg.all_instances() if selectedInstSet in instance.instsets])

In [ ]:
# Load the full betweenness scores for each instance
fullResults = dict()
for instance in cfg.all_instances():
    fullResults[instance.filename] = dict()

for run in cfg.discover_all_runs():
    algo = run.experiment.name
    variant = None if len(run.experiment.variation) == 0 else run.experiment.variation[0].name
    if algo != ('kadabra'+suffix) or variant == selectedVariant:
        instance = run.instance.filename
        
        if not algo in fullResults[instance]:
            fullResults[instance][algo] = list()

        fullFilename = run.output_file_path('out')+"-full"
        try:
            with open(fullFilename, 'r') as f:
                fullResults[instance][algo].append(numpy.loadtxt(f))
        except:
            pass

In [ ]:
# Load all running times into a single list of dictionaries
def load_file(run, f):
    data = yaml.load(f)
    return {
        'experiment': run.experiment.name,
        'variant': run.experiment.variation,
        'instance': run.instance.filename,
        'run_time': data['run_time'],
        'parameters' : data['parameters']
    }

times = cfg.collect_successful_results(load_file)

In [ ]:
for instance in cfg.all_instances():
    print(instance.filename, instance.instsets)

In [ ]:
completedInstances = dict()

for instance in cfg.all_instances():
    for run in times:
        algo = run['experiment']
        if not algo in completedInstances:
            completedInstances[algo] = set()
        
        if algo != 'kadabra'+suffix or (len(run['variant']) > 0 and run['variant'][0].name == selectedVariant):
            if run['instance'] == instance.filename:
                completedInstances[algo].add(instance)

In [ ]:
len(completedInstances['rk'+suffix])

# Significance Test Comparing Approximation Quality

In [ ]:
import scipy
import scipy.stats

In [ ]:
# Get average errors for each algorithm and instance
avgErrors = dict()

# We compare the results only on those instances from the evaluation set on which both algorithms finished
compareInstances = evaluationInstances & completedInstances['rk'+suffix] & completedInstances['kadabra'+suffix]
# Sort the instances by vertex count
sortedInstances = sorted(compareInstances, key=lambda x : len(exactValues[x.filename]) )

for instance in compareInstances:
    instanceName = instance.filename
    avgErrors[instanceName] = dict()
    size = len(exactValues[instanceName])
    for algo in fullResults[instanceName].keys():
        if len(fullResults[instanceName][algo]) == 0:
            print("No full results for " + algo + " on " + instanceName)
            continue
        if len(fullResults[instanceName][algo][0]) != size:
            print("Result for " + algo + " on " + instanceName + " has length " \
              + str(len(fullResults[instanceName][algo][0])) + " instead of " + str(size))
            continue

        diff = [abs(exactValues[instanceName][i] - fullResults[instanceName][algo][0][i]) for i in range(size)]
        avgErrors[instanceName][algo] = sum(diff)/size

In [ ]:
# Compare measured difference between average errors with the null hypothesis of no difference
rkErrors = []
kadabraErrors = []

for instance in sortedInstances:
        rkErrors.append(avgErrors[instance.filename]['rk'+suffix])
        kadabraErrors.append(avgErrors[instance.filename]['kadabra'+suffix])

scipy.stats.wilcoxon(rkErrors, kadabraErrors)

In [ ]:
[kadabraErrors[i]/rkErrors[i] for i in range(len(rkErrors))]

In [ ]:
plt.plot([kadabraErrors[i]*10**5 for i in range(len(rkErrors))])

# Bayesian Parameter Estimation for Scaling Parameter

In [ ]:
import math
import pymc3 as pm

In [ ]:
# Take logarithms of sizes and running times as preparation for scaling model
algos = ['rk'+suffix, 'kadabra'+suffix]
logTimes = dict()
for algo in algos:
    logTimes[algo] = dict()

for run in times:
    instance = run['instance']
    runtime = run['run_time']
    
    if run['experiment'] == algos[1]:
        if len(run['variant']) == 0 or not run['variant'][0].name == selectedVariant:
            continue
    
    logTimes[run['experiment']][instance] = math.log(runtime)
    
logTimesRK = []
logTimesKadabra = []
for instance in sortedInstances:
    logTimesRK.append(logTimes[algos[0]][instance.filename])
    logTimesKadabra.append(logTimes[algos[1]][instance.filename])

In [ ]:
# Define scaling model as T_{kadabra}(instance) = exp(\alpha) * T_{rk}(instance)^{\beta} + \mathcal{N}(0, \sigma)
# Parameters have vague priors to reflect our initial ignorance
basic_model = pm.Model()

with basic_model:
    
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10)
    sigma = pm.InverseGamma('sigma', alpha=1, beta=1)

    mu = alpha + beta*logTimesRK
    
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sigma, observed=logTimesKadabra)

In [ ]:
# Approximate posterior distribution of parameters with samples.
# PyMC3 uses the Markov-Chain Monte Carlo approach, constructing a Markov chain with the same 
# stationary distribution as the desired posterior distribution of the parameters.
# The number of required samples depends on the complexity of the model.

from scipy import optimize

with basic_model:
    # draw 10000 posterior samples
    trace = pm.sample(10000)

In [ ]:
# Show the posterior distribution of parameters. 
# HPD_2.5 and HPD 97.5 designate the limits of an interval containing 95% of the probability mass
pm.summary(trace)

In [ ]:
# Graphical representation of the approximated distribution of parameters.
# If the coloured lines do not match, this indicates that the distribution still depends on the starting value and 
# the markov chains have not converged sufficiently. In this case, more samples are needed.
_ = pm.traceplot(trace)

# Bayes Factor for Diameter Influence

In [ ]:
# Get sorted logsizes
logSizes = [math.log(len(exactValues[instance.filename])) for instance in sortedInstances]

In [ ]:
# Get running times of kadabra, sorted by instance size.
sortedLogTimesKadabra = []

for instance in sortedInstances:
    try:
        sortedLogTimesKadabra.append(logTimes[algo][instance.filename])
    except IndexError:
        print("No run found for " + algo + " on " + instance.filename)

In [ ]:
# Since the diameters are not stored with the instances, enter them manually from Table 1 in the paper

diameters = dict()
diameters['moreno_blogs'] = 8
diameters['petster-hamster'] = 10
diameters['ego-facebook'] = 9
diameters['openflights'] = 13
diameters['opsahl-powergrid'] = 46
diameters['p2p-Gnutella08'] = 9
diameters['advogato'] = 9
diameters['wiki-Vote'] = 7
diameters['p2p-Gnutella05'] = 9
diameters['p2p-Gnutella04'] = 10
diameters['foldoc'] = 8
diameters['twin'] = 25
diameters['cfinder-google'] = 7
diameters['ca-AstroPh'] = 14
diameters['ca-cit-HepTh'] = 9
diameters['subelj_cora'] = 20
diameters['ego-twitter'] = 15
diameters['ego-gplus'] = 8
diameters['p2p-Gnutella24'] = 11
diameters['ca-cit-HepPh'] = 9
diameters['cit-HepPh'] = 14
diameters['dblp-cite'] = 2
diameters['loc-brightkite_edges'] = 18
diameters['edit-frwikinews'] = 7
diameters['dimacs9-BAY'] = 837
diameters['dimacs9-COL'] = 1255 
diameters['roadNet-PA'] = 794
diameters['roadNet-TX'] = 1064 
diameters['facebook-wosn-wall'] = 18
diameters['edit-frwikibooks'] = 12

logDiameters = [math.log(diameters[instance.filename]) for instance in sortedInstances]
assert(len(logDiameters) == len(logSizes))

In [ ]:
# Define a hierarchical model comparing the fit of only T_RK vs T_RK+diameter to the running times.
# The two models are:
#     T_{kadabra}(instance) = exp(\alpha) * T_{RK}(instance)^{\beta} + \mathcal{N}(0, \sigma)
#     T_{kadabra}(instance) = exp(\alpha) * T_{RK}(instance)^{\beta} + diameter(instance)^{\gamma} + \mathcal{N}(0, \sigma)
# 
# The difference is the addition of the diameter term.
# Thus, multiplying this term with the boolean indicator variable selected_model serves to distinguish between the models.
basic_model = pm.Model()

with basic_model:
    
    pi = (0.5, 0.5)
    
    selected_model = pm.Bernoulli('selected_model', p=pi[1])

    alpha = pm.Normal('alpha', mu=0, sd=1)
    beta = pm.Normal('beta', mu=0, sd=1)
    gamma = pm.Normal('gamma', mu=0, sd=1)
    sigma = pm.InverseGamma('sigma', alpha=1,beta=1)
    
    mu = alpha + beta*logTimesRK + gamma*logDiameters*selected_model
    
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sigma, observed=sortedLogTimesKadabra)

In [ ]:
# Since the model is more complex since the last one, we take more samples to get accurate estimates.
with basic_model:
    # draw 200000 posterior samples
    trace = pm.sample(200000,tune=1000)

In [ ]:
# Print key values of posterior distribution
pm.summary(trace)

In [ ]:
# Calculate the Bayes Factor as posterior ratio times inverse prior ratio
BF = ((1-pm.summary(trace)['mean']['selected_model'])/pm.summary(trace)['mean']['selected_model']) * (pi[1]/pi[0])

In [ ]:
print(BF)

In [ ]:
# Plot traces.
_ = pm.traceplot(trace)